In [1]:
#Importamos las bibliotecas de tratamiento de datos
#import sqlite3
from pylab import*
import sqldf
import pandas as pd
import numpy as np
import scipy.stats
from scipy.special import gamma
import seaborn as sns
from scipy.stats import beta
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
from random import sample
from random import choices

In [42]:
#Cargamos la base de datos creada en R
Covariables=pd.read_csv("./Covariables.csv")
n=len(Covariables)

In [43]:
#Generamos de manera aleatoria un vector que toma valores en la lista de ofertas mencionada en el proyecto (dividida entre 12.97 para obtener en dólares)
Covariables["a1"]=choices([25/(12.97),50/(12.97),75/(12.97),100/(12.97),125/(12.97),150/(12.97),175/(12.97),200/(12.97)],k=n)
Covariables["a3"]=Covariables["a1"]*3

In [44]:
#Definimos el cuadrado de las cantidades anteriores
Covariables["a1cuadrada"]=(Covariables["a1"])*(Covariables["a1"])
Covariables["a3cuadrada"]=(Covariables["a3"])*(Covariables["a3"])

In [45]:
Covariables.head(10)

,Edades,ITS_o_VIH,cond_ultimavez,companeros_sex,recien_conocido,pareja_estable,trabajador_sexual,estudiante,nivel_educativo,indice_riqueza,a1,a3,a1cuadrada,a3cuadrada
0,19,0,1,1,0,1,0,0,5,0.441429,11.565150,34.695451,133.752703,1203.774323
1,22,0,1,8,1,0,0,1,2,-0.115643,15.420200,46.260601,237.782582,2140.043241
2,21,0,1,8,0,0,0,0,3,1.524031,15.420200,46.260601,237.782582,2140.043241
3,22,0,1,1,0,0,0,0,3,0.117350,3.855050,11.565150,14.861411,133.752703
4,23,0,0,6,0,0,0,1,1,0.381604,3.855050,11.565150,14.861411,133.752703
5,22,0,1,6,0,0,0,0,3,-0.454301,15.420200,46.260601,237.782582,2140.043241
6,18,0,1,3,0,0,0,1,4,0.450599,3.855050,11.565150,14.861411,133.752703
7,20,0,1,1,0,1,0,0,3,-0.308806,5.782575,17.347726,33.438176,300.943581
8,22,1,1,3,0,0,0,1,3,0.268226,11.565150,34.695451,133.752703,1203.774323
9,21,0,1,5,1,1,0,1,4,0.446898,9.637625,28.912876,92.883821,835.954391


In [46]:
#Definimos una función que nos da las probabilidades de un modelo logit,probit según los coeficientes dados. En este caso
#está ajustada para probit
def probabilidades(base,lista_nombres,lista_coeficientes,intercepto):
    baseaux=base[lista_nombres]
    baseaux["logmomio"]=intercepto
    for i in range(0,len(lista_nombres)):
        baseaux["logmomio"]=(baseaux[lista_nombres[i]]*lista_coeficientes[i])+baseaux["logmomio"] 
    #probas=np.exp(baseaux["logmomio"])/(1+np.exp(baseaux["logmomio"]))
    probas=pd.DataFrame(scipy.stats.norm(0, 1).cdf(baseaux["logmomio"]))
    return probas

In [47]:
#Una función que convierte las probas del inciso anterior en decisiones según una proba de corte dada
def decision(base,nombre,vectordeprobas,proba_corte):
    base.loc[vectordeprobas.iloc[:,0]>=proba_corte,nombre]=1
    base.loc[vectordeprobas.iloc[:,0]<proba_corte,nombre]=0
    return base

In [53]:
##Primero para el valor a1 de dinero, usando los valores del modelo presentado en el artículo. 
## Nota: las probas de cortes y los interceptos pueden variar a los mencionados en el proyecto, esto se debe a experimentos que se hicieron después de construir la base
significativasa1=["a1","a1cuadrada","ITS_o_VIH","recien_conocido"]
coeficientesa1=[0.0750,-0.0066,-0.1470,0.0622]
probacorte_a1=0.82

In [54]:
#Tomamos las decisiones respecto a a1: 
probas_a1=probabilidades(Covariables,significativasa1,coeficientesa1,0.735)

<ipython-input-46-f4f33f9912c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=intercepto
<ipython-input-46-f4f33f9912c8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=(baseaux[lista_nombres[i]]*lista_coeficientes[i])+baseaux["logmomio"]


In [55]:
probas_a1.iloc[:,0].unique()

array([0.76412002, 0.64964047, 0.62633047, 0.8227883 , 0.82843557,
       0.71654846, 0.81779361, 0.80373634, 0.70726269, 0.83722489,
       0.82049235, 0.84380054, 0.80088558, 0.76054074, 0.82145292,
       0.75734532, 0.72827544, 0.77636932, 0.78283936, 0.78202294,
       0.83848328, 0.73722658, 0.59380736, 0.78050984, 0.80598657,
       0.78843452, 0.56951911, 0.77942213, 0.79989422, 0.67745918,
       0.65483436, 0.79845525])

In [56]:
decision(Covariables,"aceptoa1",probas_a1,probacorte_a1)

,Edades,ITS_o_VIH,cond_ultimavez,companeros_sex,recien_conocido,pareja_estable,trabajador_sexual,estudiante,nivel_educativo,indice_riqueza,a1,a3,a1cuadrada,a3cuadrada,aceptoa1
0,19,0,1,1,0,1,0,0,5,0.441429,11.56515,34.695451,133.752703,1203.774323,0.0
1,22,0,1,8,1,0,0,1,2,-0.115643,15.42020,46.260601,237.782582,2140.043241,0.0
2,21,0,1,8,0,0,0,0,3,1.524031,15.42020,46.260601,237.782582,2140.043241,0.0
3,22,0,1,1,0,0,0,0,3,0.117350,3.85505,11.565150,14.861411,133.752703,1.0
4,23,0,0,6,0,0,0,1,1,0.381604,3.85505,11.565150,14.861411,133.752703,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,19,0,1,8,0,0,0,0,2,0.015663,11.56515,34.695451,133.752703,1203.774323,0.0
1149,22,0,1,2,0,0,0,0,5,0.204158,11.56515,34.695451,133.752703,1203.774323,0.0
1150,19,0,1,1,0,1,0,1,4,1.700518,7.71010,23.130301,59.445646,535.010810,1.0
1151,18,0,1,7,0,0,0,0,3,0.208716,7.71010,23.130301,59.445646,535.010810,1.0


In [57]:
#Estudiamos la proporción de personas que aceptaron la oferta a1. De nuevo, puede variar con la base que se usó en el proyecto por la nota de antes
Covariables["aceptoa1"].sum()/n

0.3954900260190807

In [74]:
#Ahora para a3 haremos en los siguientes bloques el mismo procedimiento: 
significativasa3=["a3","a3cuadrada"]
coeficientesa3=[0.0326,-0.0009]
probacorte_a3=0.835

In [75]:
probas_a3=probabilidades(Covariables,significativasa3,coeficientesa3,0.763)

<ipython-input-46-f4f33f9912c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=intercepto
<ipython-input-46-f4f33f9912c8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=(baseaux[lista_nombres[i]]*lista_coeficientes[i])+baseaux["logmomio"]


In [76]:
probas_a3.iloc[:,0].unique()

array([0.79122378, 0.63497412, 0.84605192, 0.85490083, 0.82975583,
       0.82158378, 0.72839303, 0.84979115])

In [77]:
decision(Covariables,"aceptoa3",probas_a3,probacorte_a3)

,Edades,ITS_o_VIH,cond_ultimavez,companeros_sex,recien_conocido,pareja_estable,trabajador_sexual,estudiante,nivel_educativo,indice_riqueza,a1,a3,a1cuadrada,a3cuadrada,aceptoa1,aceptoa3
0,19,0,1,1,0,1,0,0,5,0.441429,11.56515,34.695451,133.752703,1203.774323,0.0,0.0
1,22,0,1,8,1,0,0,1,2,-0.115643,15.42020,46.260601,237.782582,2140.043241,0.0,0.0
2,21,0,1,8,0,0,0,0,3,1.524031,15.42020,46.260601,237.782582,2140.043241,0.0,0.0
3,22,0,1,1,0,0,0,0,3,0.117350,3.85505,11.565150,14.861411,133.752703,1.0,1.0
4,23,0,0,6,0,0,0,1,1,0.381604,3.85505,11.565150,14.861411,133.752703,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,19,0,1,8,0,0,0,0,2,0.015663,11.56515,34.695451,133.752703,1203.774323,0.0,0.0
1149,22,0,1,2,0,0,0,0,5,0.204158,11.56515,34.695451,133.752703,1203.774323,0.0,0.0
1150,19,0,1,1,0,1,0,1,4,1.700518,7.71010,23.130301,59.445646,535.010810,1.0,1.0
1151,18,0,1,7,0,0,0,0,3,0.208716,7.71010,23.130301,59.445646,535.010810,1.0,1.0


In [78]:
Covariables["aceptoa3"].sum()/n

0.37814397224631396

In [79]:
#Ahora crearemos las columnas a2 y a4 dependiendo de si se aceptó o no a1 y a3 
Covariables.loc[Covariables["aceptoa1"]==1,"a2"]=Covariables["a1"]*0.7
Covariables.loc[Covariables["aceptoa1"]==0,"a2"]=(Covariables["a1"]+(200/12.97))*0.5

In [80]:
Covariables.loc[Covariables["aceptoa3"]==1,"a4"]=Covariables["a3"]*0.7
Covariables.loc[Covariables["aceptoa3"]==0,"a4"]=(Covariables["a3"])*2

In [81]:
#Columnas con los cuadrados de a2 y a4
Covariables["a2cuadrada"]=(Covariables["a2"])*(Covariables["a2"])
Covariables["a4cuadrada"]=(Covariables["a4"])*(Covariables["a4"])

In [82]:
#Ahora, a tomar decisiones sobre si aceptar las nuevas ofertas igual que antes, con los mismos procedimientos:

#a2
significativasa2=["a2","a2cuadrada","companeros_sex","trabajador_sexual"]
coeficientesa2=[0.0519,-0.0069,0.0137,-0.1735]
probacorte_a2=0.774

In [83]:
probas_a2=probabilidades(Covariables,significativasa2,coeficientesa2,0.788)

<ipython-input-46-f4f33f9912c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=intercepto
<ipython-input-46-f4f33f9912c8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=(baseaux[lista_nombres[i]]*lista_coeficientes[i])+baseaux["logmomio"]


In [84]:
probas_a2.iloc[:,0].unique()

array([0.59708498, 0.52281048, 0.81367153, 0.83147429, 0.51189014,
       0.82092803, 0.81560082, 0.60765374, 0.66408302, 0.82448868,
       0.6886871 , 0.80778033, 0.54378255, 0.83564665, 0.68392719,
       0.49549547, 0.80863177, 0.8561377 , 0.84661582, 0.49003091,
       0.71712854, 0.81881169, 0.78048901, 0.65403318, 0.67893843,
       0.52928307, 0.53915637, 0.81811958, 0.81922787, 0.82877162,
       0.55543636, 0.65907221, 0.56084368, 0.61335258, 0.82526613,
       0.62854759, 0.63415104, 0.69830522, 0.85827967, 0.61290902,
       0.4965164 , 0.59222327, 0.77230092, 0.80832003, 0.58155291,
       0.64896673, 0.8390161 , 0.61814369, 0.8469797 , 0.78850532,
       0.79634757, 0.82634671, 0.48456821, 0.64906933, 0.53971234,
       0.82280987, 0.67411659, 0.82674118, 0.74381306, 0.771318  ,
       0.61858546, 0.5500187 , 0.81963248, 0.82800413, 0.76814326,
       0.66906483, 0.63371507, 0.60282565, 0.52826475, 0.64397678,
       0.82983831, 0.62335684, 0.68382874, 0.56543663, 0.83223

In [85]:
decision(Covariables,"aceptoa2",probas_a2,probacorte_a2)

,Edades,ITS_o_VIH,cond_ultimavez,companeros_sex,recien_conocido,pareja_estable,trabajador_sexual,estudiante,nivel_educativo,indice_riqueza,...,a3,a1cuadrada,a3cuadrada,aceptoa1,aceptoa3,a2,a4,a2cuadrada,a4cuadrada,aceptoa2
0,19,0,1,1,0,1,0,0,5,0.441429,...,34.695451,133.752703,1203.774323,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0
1,22,0,1,8,1,0,0,1,2,-0.115643,...,46.260601,237.782582,2140.043241,0.0,0.0,15.420200,92.521203,237.782582,8560.172963,0.0
2,21,0,1,8,0,0,0,0,3,1.524031,...,46.260601,237.782582,2140.043241,0.0,0.0,15.420200,92.521203,237.782582,8560.172963,0.0
3,22,0,1,1,0,0,0,0,3,0.117350,...,11.565150,14.861411,133.752703,1.0,1.0,2.698535,8.095605,7.282092,65.538824,1.0
4,23,0,0,6,0,0,0,1,1,0.381604,...,11.565150,14.861411,133.752703,1.0,1.0,2.698535,8.095605,7.282092,65.538824,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,19,0,1,8,0,0,0,0,2,0.015663,...,34.695451,133.752703,1203.774323,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0
1149,22,0,1,2,0,0,0,0,5,0.204158,...,34.695451,133.752703,1203.774323,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0
1150,19,0,1,1,0,1,0,1,4,1.700518,...,23.130301,59.445646,535.010810,1.0,1.0,5.397070,16.191210,29.128366,262.155297,1.0
1151,18,0,1,7,0,0,0,0,3,0.208716,...,23.130301,59.445646,535.010810,1.0,1.0,5.397070,16.191210,29.128366,262.155297,1.0


In [86]:
Covariables["aceptoa2"].sum()/n

0.4588031222896791

In [87]:
#a4
significativasa4=["a4"]
coeficientesa4=[-0.0085]
probacorte_a4=0.804

In [88]:
probas_a4=probabilidades(Covariables,significativasa4,coeficientesa4,1.1)

<ipython-input-46-f4f33f9912c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=intercepto
<ipython-input-46-f4f33f9912c8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseaux["logmomio"]=(baseaux[lista_nombres[i]]*lista_coeficientes[i])+baseaux["logmomio"]


In [89]:
probas_a4.iloc[:,0].unique()

array([0.69503638, 0.62307609, 0.84877352, 0.8405646 , 0.72856578,
       0.84175483, 0.65978395, 0.83206929])

In [90]:
decision(Covariables,"aceptoa4",probas_a4,probacorte_a4)

,Edades,ITS_o_VIH,cond_ultimavez,companeros_sex,recien_conocido,pareja_estable,trabajador_sexual,estudiante,nivel_educativo,indice_riqueza,...,a1cuadrada,a3cuadrada,aceptoa1,aceptoa3,a2,a4,a2cuadrada,a4cuadrada,aceptoa2,aceptoa4
0,19,0,1,1,0,1,0,0,5,0.441429,...,133.752703,1203.774323,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0,0.0
1,22,0,1,8,1,0,0,1,2,-0.115643,...,237.782582,2140.043241,0.0,0.0,15.420200,92.521203,237.782582,8560.172963,0.0,0.0
2,21,0,1,8,0,0,0,0,3,1.524031,...,237.782582,2140.043241,0.0,0.0,15.420200,92.521203,237.782582,8560.172963,0.0,0.0
3,22,0,1,1,0,0,0,0,3,0.117350,...,14.861411,133.752703,1.0,1.0,2.698535,8.095605,7.282092,65.538824,1.0,1.0
4,23,0,0,6,0,0,0,1,1,0.381604,...,14.861411,133.752703,1.0,1.0,2.698535,8.095605,7.282092,65.538824,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,19,0,1,8,0,0,0,0,2,0.015663,...,133.752703,1203.774323,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0,0.0
1149,22,0,1,2,0,0,0,0,5,0.204158,...,133.752703,1203.774323,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0,0.0
1150,19,0,1,1,0,1,0,1,4,1.700518,...,59.445646,535.010810,1.0,1.0,5.397070,16.191210,29.128366,262.155297,1.0,1.0
1151,18,0,1,7,0,0,0,0,3,0.208716,...,59.445646,535.010810,1.0,1.0,5.397070,16.191210,29.128366,262.155297,1.0,1.0


In [91]:
Covariables["aceptoa4"].sum()/n

0.5056374674761491

In [92]:
#Ahora sumamos todas las respuestas a las ofertas (que son variables binarias)
Covariables["suma"]=Covariables["aceptoa1"]+Covariables["aceptoa2"]+Covariables["aceptoa3"]+Covariables["aceptoa4"]

In [93]:
#Según si la suma es o no positiva, obtenemos la respuesta que deseábamos
Covariables.loc[Covariables["suma"]>0,"aceptar_intervencion"]=1
Covariables.loc[Covariables["suma"]==0,"aceptar_intervencion"]=0


In [94]:
Covariables.head(35)

,Edades,ITS_o_VIH,cond_ultimavez,companeros_sex,recien_conocido,pareja_estable,trabajador_sexual,estudiante,nivel_educativo,indice_riqueza,...,aceptoa1,aceptoa3,a2,a4,a2cuadrada,a4cuadrada,aceptoa2,aceptoa4,suma,aceptar_intervencion
0,19,0,1,1,0,1,0,0,5,0.441429,...,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0,0.0,0.0,0.0
1,22,0,1,8,1,0,0,1,2,-0.115643,...,0.0,0.0,15.420200,92.521203,237.782582,8560.172963,0.0,0.0,0.0,0.0
2,21,0,1,8,0,0,0,0,3,1.524031,...,0.0,0.0,15.420200,92.521203,237.782582,8560.172963,0.0,0.0,0.0,0.0
3,22,0,1,1,0,0,0,0,3,0.117350,...,1.0,1.0,2.698535,8.095605,7.282092,65.538824,1.0,1.0,4.0,1.0
4,23,0,0,6,0,0,0,1,1,0.381604,...,1.0,1.0,2.698535,8.095605,7.282092,65.538824,1.0,1.0,4.0,1.0
5,22,0,1,6,0,0,0,0,3,-0.454301,...,0.0,0.0,15.420200,92.521203,237.782582,8560.172963,0.0,0.0,0.0,0.0
6,18,0,1,3,0,0,0,1,4,0.450599,...,1.0,1.0,2.698535,8.095605,7.282092,65.538824,1.0,1.0,4.0,1.0
7,20,0,1,1,0,1,0,0,3,-0.308806,...,1.0,1.0,4.047803,12.143408,16.384706,147.462355,1.0,1.0,4.0,1.0
8,22,1,1,3,0,0,0,1,3,0.268226,...,0.0,0.0,13.492675,69.390902,182.052290,4815.097292,0.0,0.0,0.0,0.0
9,21,0,1,5,1,1,0,1,4,0.446898,...,0.0,0.0,12.528913,57.825752,156.973658,3343.817564,0.0,0.0,0.0,0.0


In [95]:
#Imprimimos la base (Nota, este código no regresará la base usada en el código Aplicacion_modelos.ipynb, pero esa base se subió junto a lo demás. )
Covariables.to_csv("Base_de_regresion3.csv")